# Midterm - Hung Le

## FINM 25000 - 2024

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

# Instructions

## Please note the following:

Points
* The exam is `115` points.
* You have `180` minutes to complete the exam.
* For every minute late you submit the exam, you will lose one point.

Submission
* You will upload your solution to the `Midterm` assignment on Canvas. (Be sure to **submit** on Canvas, not just **save** on Canvas.
* Your submission should be readable, (the graders can understand your answers,) and it should **include all code used in your analysis in a file format that the code can be executed.** 

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo--you are welcome to use the code posted there without citation.

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:
* `midterm_data.xlsx`

This file has sheets for...
* `info` - names of each stock ticker
* `excess returns` - weekly excess returns on several stocks
* `SPY` - weekly excess returns on SPY
* `forecasting` - monthly data on `USO` asset returns and two forecasting signals.

Note 
* the data for `excess returns` and `SPY` is **weekly** so any annualizations should use `52` weeks in a year.
* the data for `forecasting` is monthly, so any annualization should use `12` months in a year.

#### If useful
here is code to load in the data.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels as sm
from functools import partial
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 9)
pd.options.display.float_format = '{:.4f}'.format

In [2]:
FILEIN = 'midterm_data.xlsx'
sheet_exrets = 'excess returns'
sheet_spy = 'spy'
sheet_forecasting = 'forecasting'

retsx = pd.read_excel(FILEIN, sheet_name=sheet_exrets).set_index('date')
spy = pd.read_excel(FILEIN, sheet_name=sheet_spy).set_index('date')
forecasting = pd.read_excel(FILEIN, sheet_name=sheet_forecasting).set_index('date')

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 50     |
| 2       | 25     |
| 3       | 20     |
| 4       | 20     |

### Each numbered question is worth 5 points.

### Notation
(Hidden LaTeX commands)

$$\newcommand{\mux}{\tilde{\boldsymbol{\mu}}}$$
$$\newcommand{\wtan}{\boldsymbol{\text{w}}^{\text{tan}}}$$
$$\newcommand{\wtarg}{\boldsymbol{\text{w}}^{\text{port}}}$$
$$\newcommand{\mutarg}{\tilde{\boldsymbol{\mu}}^{\text{port}}}$$
$$\newcommand{\wEW}{\boldsymbol{\text{w}}^{\text{EW}}}$$
$$\newcommand{\wRP}{\boldsymbol{\text{w}}^{\text{RP}}}$$
$$\newcommand{\wREG}{\boldsymbol{\text{w}}^{\text{REG}}}$$
$$\newcommand{\targ}{\text{USO}}$$

# 1. Short Answer

### No Data Needed

These problem does not require any data file. Rather, analyze the situation conceptually, based on the information below. 

### 1

In what sense was ProShares `HDG` successful in hedging the `HFRI`, and in what sense was it unsuccessful in tracking the `HFRI`?

It was successful in hedging because of a positive alpha, and relatively ok information ratio, which is essentially the Sharpe of the hedged position. It was unsuccessful in tracking the `HFRI`, because of the relatively high tracking error, which is standard deviation of $\epsilon_t$, and that there remains an outstanding $\alpha$.

### 2

Did we find that **TIPS** have been useful in expanding the mean-variance frontier in the past? Did we conclude they might be useful in the future? Explain.

We found that TIPS has not been so useful in expanding the MV frontier in the past, with high correlations with IEF and BWX, and the target portfolio with no TIPS performing with roughly the same vol (hence same Sharpe) as the target portfolio with TIPS. This might change in the future, though, because past performances are not necessarily indicative of future performances and thus the high correlations might decrease, enabling TIPS to expand the MV frontier.

### 3.

Consider a Linear Factor Pricing Model (LFPM).

Which metric do we examine to understand its fit, (or errors)?

We examine if $|\alpha|$ is high.

### 4.

What aspect of the classic mean-variance optimization approach leads to extreme answers? How did regularization help with this issue?

Inverting the covariance matrix leads to extreme answers. Regularization helps with stabilizing this inverting procedure, so that small changes in the covariance matrix doesn't produce large changes in its inverse.

### 5.

Suppose investors are **not** mean-variance investors. If we find an investment with a Sharpe ratio higher than the "market", would this would be inconsistent with the CAPM?

Yes, because the CAPM suggests that the market has the highest Sharpe, $SR(r^i) = \rho_{i, m} SR(r^m)$ and the correlation $\rho_{i, m}$ is always $\leq 1$.

### 6.

Which is more useful in assessing the model’s fit for pricing: the r-squared of the time-series regressions, the r-squared of the cross-sectional regression, or neither?

Neither. What matters is the $\alpha$ of the cross-sectional regression.

### 7.

GMO stated that they had a “contrarian” investment style. What did they mean by this? Was this seen in our investigation of the fund, GMWAX?

GMO believed that actual market prices could deviate from fundamental value, and that investors overreact to short-term trends, and that the fund can identify these bubbles and invest in long-term trends. This is reflected in our investigation with high market betas, i.e., large exposure to systematic risk.

### 8.

How does Harvard make their portfolio allocation more realistic than a basic mean- variance optimization would imply? Is their approach easily implemented and computed from a numerical standpoint?

Harvard had bounds on their asset positions. This is still easily implemented and computed from a numerical standpoint.

### 9.

If we want to hedge a portfolio's returns with respect to SPY, how could we calculate the optimal ratio? How would this ratio then be used to build the hedged position?

Regress against SPY as $r^p = \alpha + \beta_{p, SPY} r^{SPY} + \epsilon$, then have the hedged position $r^p - \beta_{p, SPY} r^{SPY}$.

### 10.

Name one way in which Fama and French construct the factors that helps reduce cross-factor correlation.

For each factor, they long some and short some; for example, for `SMB`, they long small cap companies and short large cap companies. So the overall trend of the market is neutralized, and thus cross-factor correlation between, say, `SMB` and `HML`, simply because of the overall market movement, is reduced.

***

# 2. Allocation


Consider a mean-variance optimization of **excess** returns provided in `midterm_data.xlsx.`

In [3]:
def calc_return_metrics(data, as_df=True, adj = 52):
    """
    Calculate return metrics for a DataFrame of assets.

    Args:
        data (pd.DataFrame): DataFrame of asset returns.
        as_df (bool, optional): Return a DF or a dict. Defaults to False (return a dict).
        adj (int, optional): Annualization. Defaults to 12.

    Returns:
        Union[dict, DataFrame]: Dict or DataFrame of return metrics.
    """
    summary = dict()
    summary["Annualized Return"] = data.mean() * adj
    summary["Annualized Volatility"] = data.std() * np.sqrt(adj)
    summary["Annualized Sharpe Ratio"] = (
        summary["Annualized Return"] / summary["Annualized Volatility"]
    )
    # summary["Annualized Sortino Ratio"] = summary["Annualized Return"] / (
    #     data[data < 0].std() * np.sqrt(adj)
    # )
    return pd.DataFrame(summary, index=data.columns) if as_df else summary


def calc_risk_metrics(data, as_df=True, var=0.05):
    """
    Calculate risk metrics for a DataFrame of assets.

    Args:
        data (pd.DataFrame): DataFrame of asset returns.
        as_df (bool, optional): Return a DF or a dict. Defaults to False.
        adj (int, optional): Annualizatin. Defaults to 12.
        var (float, optional): VaR level. Defaults to 0.05.

    Returns:
        Union[dict, DataFrame]: Dict or DataFrame of risk metrics.
    """
    summary = dict()
    summary["Skewness"] = data.skew()
    summary["Kurtosis"] = data.kurtosis()
    summary[f"VaR ({var})"] = data.quantile(var, axis=0)
    summary[f"CVaR ({var})"] = data[data <= data.quantile(var, axis=0)].mean()
    summary["Min"] = data.min()
    summary["Max"] = data.max()

    wealth_index = 1000 * (1 + data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks) / previous_peaks

    summary["Max Drawdown"] = drawdowns.min()

    summary["Bottom"] = drawdowns.idxmin()
    summary["Peak"] = previous_peaks.idxmax()

    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][: drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin() :]]).T
        recovery_date.append(
            recovery_wealth[recovery_wealth[col] >= prev_max].index.min()
        )
    summary["Recovery"] = ["-" if pd.isnull(i) else i for i in recovery_date]

    summary["Duration (days)"] = [
        (i - j).days if i != "-" else "-"
        for i, j in zip(summary["Recovery"], summary["Bottom"])
    ]

    return pd.DataFrame(summary, index=data.columns) if as_df else summary


def calc_performance_metrics(data, adj = 52, var=0.05):
    """
    Aggregating function for calculating performance metrics. Returns both
    risk and performance metrics.

    Args:
        data (pd.DataFrame): DataFrame of asset returns.
        adj (int, optional): Annualization. Defaults to 12.
        var (float, optional): VaR level. Defaults to 0.05.

    Returns:
        DataFrame: DataFrame of performance metrics.
    """
    summary = {
        **calc_return_metrics(data=data, as_df = False, adj=adj),
        **calc_risk_metrics(data=data, as_df = False, var=var),
    }
    summary["Calmar Ratio"] = summary["Annualized Return"] / abs(
        summary["Max Drawdown"]
    )
    return pd.DataFrame(summary, index=data.columns)

def plot_correlation_matrix(corrs, ax=None):
    if ax:
        sns.heatmap(
            corrs,
            annot=True,
            cmap="coolwarm",
            vmin=-1,
            vmax=1,
            linewidths=0.7,
            annot_kws={"size": 10},
            fmt=".2f",
            square=True,
            cbar_kws={"shrink": 0.75},
            ax=ax,
        )
    # Correlation helper function.
    else:
        ax = sns.heatmap(
            corrs,
            annot=True,
            cmap="coolwarm",
            vmin=-1,
            vmax=1,
            linewidths=0.7,
            annot_kws={"size": 10},
            fmt=".2f",
            square=True,
            cbar_kws={"shrink": 0.75},
        )

    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment="right")
    return ax


def print_max_min_correlation(corrs):
    # Correlation helper function. Prints the min/max/absolute value
    # for the correlation matrix.
    corr_series = corrs.unstack()
    corr_series = corr_series[corr_series != 1]

    max_corr = corr_series.abs().agg(["idxmax", "max"]).T
    min_corr = corr_series.abs().agg(["idxmin", "min"]).T
    min_corr_raw = corr_series.agg(["idxmin", "min"]).T
    max_corr, max_corr_val = max_corr["idxmax"], max_corr["max"]
    min_corr, min_corr_val = min_corr["idxmin"], min_corr["min"]
    min_corr_raw, min_corr_raw_val = min_corr_raw["idxmin"], min_corr_raw["min"]

    print(
        f"Max Corr (by absolute value): {max_corr[0]} and {max_corr[1]} with a correlation of {max_corr_val:.2f}"
    )
    print(
        f"Min Corr (by absolute value): {min_corr[0]} and {min_corr[1]} with a correlation of {min_corr_val:.2f}"
    )
    print(
        f"Min Corr (raw): {min_corr_raw[0]} and {min_corr_raw[1]} with a correlation of {min_corr_raw_val:.2f}"
    )

### 1. 

Report the following **annualized** statistics:
* mean
* volatility
* Sharpe ratio

In [4]:
performance = calc_performance_metrics(retsx, adj = 52)[['Annualized Return', 'Annualized Volatility', 'Annualized Sharpe Ratio']]
display(performance)

,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio
AAPL,0.3194,0.2839,1.1252
MSFT,0.2881,0.2402,1.1993
AMZN,0.2395,0.3104,0.7715
NVDA,0.6507,0.4681,1.3900
GOOGL,0.1933,0.2742,0.7050
TSLA,0.5697,0.6070,0.9386
XOM,0.1242,0.3116,0.3986


### 2.

Report the weights of the tangency portfolio.

In [5]:
def calc_mv_portfolio(df, target = None):
    mean_rets = df.mean()
    cov_matrix = df.cov()
    inv_cov = np.linalg.inv(cov_matrix)
    ones = np.ones(mean_rets.shape)
    w_tan = (inv_cov @ mean_rets) / (ones.T @ (inv_cov @ mean_rets))
    if target is None:
        return pd.DataFrame({"Tangency Portfolio": w_tan}, index=df.columns)

    return pd.DataFrame({
        "Tangency Portfolio": w_tan,
        "Target Portfolio": w_tan * target/(mean_rets @ w_tan)
    }, index=df.columns)

In [6]:
w_tan = calc_mv_portfolio(retsx)
display(w_tan)

,Tangency Portfolio
AAPL,0.3226
MSFT,0.7875
AMZN,-0.2286
NVDA,0.4960
GOOGL,-0.5027
TSLA,0.1060
XOM,0.0193


### 3.
Report the Sharpe ratio achieved by the tangency portfolio over this sample. Annualize it (accounting for weekly data.)

In [7]:
calc_return_metrics(retsx @ w_tan, adj = 52)

,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio
Tangency Portfolio,0.5635,0.3584,1.5724


### 4.

* What weight is given to the asset with the lowest Sharpe ratio?
* What Sharpe ratio does the lowest (most negative) weight asset have?

Explain why the weights are not most extreme for the assets with the largest/smallest Share Ratios.

In [8]:
performance['Tangency Portfolio'] = w_tan
display(performance.sort_values('Annualized Sharpe Ratio'))

,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio,Tangency Portfolio
XOM,0.1242,0.3116,0.3986,0.0193
GOOGL,0.1933,0.2742,0.7050,-0.5027
AMZN,0.2395,0.3104,0.7715,-0.2286
TSLA,0.5697,0.6070,0.9386,0.1060
AAPL,0.3194,0.2839,1.1252,0.3226
MSFT,0.2881,0.2402,1.1993,0.7875
NVDA,0.6507,0.4681,1.3900,0.4960


- Weight 0.0193 is given to `XOM` with the lowest Sharpe of 0.3986.
- `GOOGL` has most negative weight at -0.5027, with Sharpe 0.7050.
- Because in an MV portfolio, what matters is the covariance between the assets, not individual asset's Sharpe.

### 5.

To target a mean return of `0.001` weekly, would you be invested in the risk-free rate or borrowing from the risk-free rate?

In [9]:
w_targeted = calc_mv_portfolio(retsx, 0.001)
display(w_targeted)
print(f"Target Portfolio Total Position: {w_targeted['Target Portfolio'].sum()}")

,Tangency Portfolio,Target Portfolio
AAPL,0.3226,0.0298
MSFT,0.7875,0.0727
AMZN,-0.2286,-0.0211
NVDA,0.4960,0.0458
GOOGL,-0.5027,-0.0464
TSLA,0.1060,0.0098
XOM,0.0193,0.0018


Target Portfolio Total Position: 0.09228463054305538


Only 9% will be invested in these stocks, and the rest should be invested in risk-free rate.

***

# 3. Performance

### 1. 

Report the following performance metrics of excess returns for Tesla (`TSLA`).
* skewness
* kurtosis

You are not annualizing any of these stats.

What do these metrics indicate about the nature of the returns?

In [10]:
calc_risk_metrics(retsx[['TSLA']])[['Skewness', 'Kurtosis']]

,Skewness,Kurtosis
TSLA,0.4415,1.5274


The positive skew means a longer positive tail. The kurtosis of around 1.52 suggests that the tails are relatively flat.

### 2. 

Report the maximum drawdown for `TSLA` over the sample.
* Ignore that your data is in excess returns rather than total returns.
* Simply proceed with the excess return data for this calculation.

In [11]:
calc_risk_metrics(retsx[['TSLA']])[['Max Drawdown']]

,Max Drawdown
TSLA,-0.6822


### 3.

For `TSLA`, calculate the following metrics, relative to `SPY`:
* market beta
* alpha
* information ratio

Annualize alpha and information ratio.

*Recall that this is weekly data, with 52 weeks per year.*

In [12]:
def unitarget_regression(y, X,  multi_X = True, adj = 52):
    """
        Regress y against univariate/multivariate X
        y: pd.Series
        X: pd.DataFrame
        multi_X: Boolean
    """
    summary = {}
    reg = LinearRegression()
    reg.fit(X, y)
    summary['Annualized Alpha'] = reg.intercept_ * adj
    summary['Rsquared'] = reg.score(X, y)
    for i, x in enumerate(X.columns):
        summary[f'Beta {x}'] = reg.coef_[i]
    eps = y - reg.predict(X)
    summary['Annualized Information Ratio'] = reg.intercept_/eps.std() * np.sqrt(adj)
    summary['Annualized Tracking Error'] = eps.std() * np.sqrt(adj)
    if not multi_X:
        name_x = X.columns[0]
        summary['Annualized Treynor Ratio'] = y.mean()/summary[f'Beta {name_x}'] * adj
    return pd.DataFrame(summary, index = [y.name])

def multitarget_regression(y, X, multi_X = False, adj = 52):
    """
        y: pd.DataFrame
        X: pd.DataFrame
        multi_X: Boolean
    """
    return pd.concat([unitarget_regression(y[col], X, multi_X, adj) for col in y.columns], axis = 0)

def regression(y, X, multi_y = False, multi_X = False, adj = 52):
    """
        multi_y: Boolean
        multi_X: Boolean
    """
    if multi_y:
        return multitarget_regression(y, X, multi_X, adj)
    return unitarget_regression(y, X, multi_X, adj)

In [13]:
regression(retsx['TSLA'], spy)[['Annualized Alpha', 'Beta SPY', 'Annualized Information Ratio']]

,Annualized Alpha,Beta SPY,Annualized Information Ratio
TSLA,0.3095,1.7768,0.5961


### 4.

Comment on what you conclude about `TSLA` based on the statistics calculated in the previous question.

The high market beta implies that it has high risk premium.

***

# 4. Hedging

### 1. 

Consider the following scenario: you are holding a \$100 million long position in `NVDA`. You wish to hedge the position using some combination of 
* `AAPL`
* `AMZN`
* `GOOGL`
* `MSFT`

Report the positions you would hold of those 4 securities for an optimal hedge.

Note:
* In the regression estimation, include an intercept.
* Use the full-sample regression. No need to worry about in-sample versus out-of-sample.

In [14]:
regression(retsx['NVDA'], retsx[['AAPL', 'AMZN', 'GOOGL', 'MSFT']], multi_X = True)

,Annualized Alpha,Rsquared,Beta AAPL,Beta AMZN,Beta GOOGL,Beta MSFT,Annualized Information Ratio,Annualized Tracking Error
NVDA,0.2738,0.4582,0.3417,0.4173,-0.0078,0.5879,0.7945,0.3446


My total position would then be
- $100M long NVDA
- $34.17M short AAPL
- $41.73M short AMZN
- $0.78M long GOOGL
- $58.79M short MSFT

### 2.

How well does the hedge do? Cite a regression statistic to support your answer.

Also estimate the volatility of the basis, (epsilon.)

The volatility of epsilon is reported above. The hedge does relatively ok, capturing positive $\alpha$. Its annualized Sharpe is the annualized Information Ratio, which is relatively ok at 0.7945.

***

# 5. Forecasting

Forecast (total) returns on oil as tracked by the ETF ticker, `USO`. 

As signals, use two interest rate signals, as seen in Treasury-notes. (No need to consider anything specific about Treasury notes, just read these as macroeconomic signals.)
* T-note rate
* month-over-month change in the T-note rate

Find the all data needed for this problem in the sheet `forecasting`.

Note that the data in this sheet is monthly, not weekly.

### 1.

Estimate a forecasting regression of $\targ$ on the two (lagged) signals.

$$r_{t+1}^{\targ} = \alpha + \beta^{x}x_t + \beta^z z_t + \epsilon_{t+1}$$

where
* $x$ denotes the interest-rate signal.
* $z$ denotes the change in rate signal.

Report the r-squared, as well as the OLS estimates for the intercept and the two betas. (No need to annualize the stats.)

In [15]:
def lagged(y, X, delta_shift = 1):
    """
        y: pd.Series
        X: pd.DataFrame
    """
    X = X.shift(delta_shift).dropna()
    y = y.loc[X.index]
    reg = LinearRegression()
    reg.fit(X, y)
    stats= {}
    stats['Rsquared'] = reg.score(X, y)
    stats['Alpha'] = reg.intercept_
    for i, x in enumerate(X.columns):
        stats[f'Beta {x}'] = reg.coef_[i]
    yhat = reg.predict(X)
    w = 0.5 + 0.5 * yhat
    X_name = ', '.join(X.columns)
    strategy_rets = (w * y).rename(X_name)
    return {
        'stats': pd.DataFrame(stats, index = [X_name]),
        'strategy_rets': pd.DataFrame(strategy_rets),
        'yhat': yhat
    }

lagged_results = lagged(forecasting['USO'], forecasting[['Tnote rate', 'Tnote rate change']])

In [16]:
lagged_results['stats']

,Rsquared,Alpha,Beta Tnote rate,Beta Tnote rate change
"Tnote rate, Tnote rate change",0.0281,0.0201,-0.0096,0.0741


### 2.

Use your forecasted returns, $\hat{r}^{\targ}_{t+1}$ to build trading weights:

$$w_t = 0.50 + 50\;\hat{r}^{\targ}_{t+1}$$

(So the rule says to hold 50% in the ETF plus/minus 50x the forecast. Recall the forecast is a monthly percentage, so it is a small number.)

Calculate the return from implementing this strategy. Denote this as $r^x_t$.

Report the first and last 5 values.

In [17]:
strategy_rets = lagged_results['strategy_rets']
display(strategy_rets.head())
display(strategy_rets.tail())

,"Tnote rate, Tnote rate change"
date,
2009-06-30,0.0213
2009-07-31,-0.0146
2009-08-31,-0.0102
2009-09-30,0.0019
2009-10-31,0.0424


,"Tnote rate, Tnote rate change"
date,
2023-07-31,0.0755
2023-08-31,0.0128
2023-09-30,0.0383
2023-10-31,-0.0365
2023-11-30,-0.0166


### 3.

Calculate the following (annualized) performance metrics for both the passive investment, $r^\targ$, as well as the strategy implemented in the previous problem, $r^x$.

* mean
* volatility
* max drawdown

**Remember to annualize mean and volatility for monthly data.** (No need to annualize max drawdown.)

**Interpretation:** For USO, calculate from 05-31-2009, while the strategy only has returns from 06-30-2009 onwards, so the timeframes are slightly different.

In [18]:
calc_performance_metrics(forecasting[['USO']], adj = 12)[['Annualized Return', 'Annualized Volatility', 'Annualized Sharpe Ratio', 'Max Drawdown']]

,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio,Max Drawdown
USO,-0.0042,0.3626,-0.0116,-0.9471


In [19]:
calc_performance_metrics(strategy_rets, adj = 12)[['Annualized Return', 'Annualized Volatility', 'Annualized Sharpe Ratio', 'Max Drawdown']]

,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio,Max Drawdown
"Tnote rate, Tnote rate change",-0.0097,0.1773,-0.0546,-0.7065


### 4.

Comment on whether the active strategy (using forecasting), $r^z$ is an improvement on the passive strategy of just holding $\targ$ directly.

The active strategy has less vol but even more negative mean, which results in a worse off Sharpe (-0.05 < -0.01). So it is not really an improvement over the passive strategy. One good thing though is that it has better max drawdown.